In [2]:
%load_ext cython

In [59]:
%%cython -a -f -c=-DCYTHON_TRACE=1

cimport cython
import numpy as np
cimport numpy as np
from libc.math cimport sqrt as c_sqrt
from libc.math cimport log as c_log

from libc.stdlib cimport rand as c_rand 
from libc.stdlib cimport RAND_MAX

cdef double c_max(double x):
    return x if x > 0.0 else 0.0
            
@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
def heston_step(float S0, float nu0, float kappa, float theta, float xi, float rho, float dt, float r, float q, int num_paths, int num_steps,
                double[:] dW_S, double[:] dW_X):
    cdef double[:] paths = np.full((num_paths*num_steps), S0, dtype='f8', order='C')
    cdef double[:] nu = np.full((num_paths), nu0, dtype='f8', order='C')
    cdef double dW_nu
    cdef Py_ssize_t t, path, idx
    for t in range(1,num_steps):
        for path in range(num_paths):
            idx = num_steps*path + t
            dW_nu = rho*dW_S[idx] + c_sqrt(1 - rho**2)*dW_X[idx]
            nu[path] = c_max(nu[path] + kappa*(theta - nu[path])*dt + xi*c_sqrt(nu[path]*dt)*dW_nu)
            paths[idx] = paths[idx-1]*(1 + (r-q)*dt + c_sqrt(nu[path]*dt)*dW_S[idx])
    return paths

In [62]:
num_paths = 100000; num_steps = 150; dt = 1/150
dW_S = np.random.normal(size=(num_paths*num_steps))
dW_X = np.random.normal(size=(num_paths*num_steps))
    
def test():
    S0 = 100.00; r = 0; q = 0; nu0 = 0.16**2; kappa = 1.0; theta = 0.16**2; xi = 0.30; rho = -0.60

    paths = np.array(heston_step(S0, nu0, kappa, theta, xi, rho, dt, r, q, num_paths, num_steps, dW_S, dW_X))
    print(np.mean(np.maximum(90.0 - paths[np.arange(1,num_paths)*num_steps-1],0)))

In [63]:
%%timeit -r 5 -n 1
test() 

4.775572751813702
4.775572751813702
4.775572751813702
4.775572751813702
4.775572751813702
547 ms ± 36.1 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
